In [2]:
import airpy as ap
import numpy as np
import pandas as pd
import gensim
import nltk
from nltk.corpus import stopwords
from gensim import corpora
from gensim import models
import pyLDAvis.gensim
import csv
import feather
import random

In [3]:
#get common names list from disk
path_names = '~/data/first_message_text_names.csv'
common_names = list(pd.read_csv(path_names)['dim_first_name'])

#get message text from disk
path_messages = '~/data/first_message_text_lower.csv'
df1 = pd.DataFrame(pd.read_csv(path_messages)['message_text_lower'].dropna())

In [4]:
#build documents list (remember to drop na's)
documents = []
for index, row in df1.iterrows():
    documents.append(row.message_text_lower)
    
#generate a stoplist
stoplist = stopwords.words('english')
stoplist += common_names

print 'common names done'

# tokenize the text 
doc_token = [[word for word in list(gensim.utils.tokenize(document)) if word not in stoplist] for document in documents]

print 'tokenization done'

# build it into dictionary map
dictionary = corpora.Dictionary(doc_token)

# create the corpus for LDA 
corpus = [dictionary.doc2bow(message_text) for message_text in list(doc_token)]

# convert to TD-IDF
tfidf = models.TfidfModel(corpus)
corpus_tfidf = tfidf[corpus]


common names done
tokenization done


In [ ]:
# train a LDA model
%time 

lda_model_30 = gensim.models.LdaModel(corpus_tfidf, num_topics=30, alpha=0.0001, id2word=dictionary, passes=6)
lda_model_30.save('lda_test.model')

In [1]:
lda_model_30 = models.LdaModel.load('lda_test.model')
cx_data =  pyLDAvis.gensim.prepare(lda_model_30, corpus_tfidf, dictionary)
pyLDAvis.display(cx_data)

NameError: name 'models' is not defined

In [29]:
mendy = lda_model_30[corpus[13231]]

In [31]:
documents[13231]

"hey! my names noah and i'm a musician. coming to la for 10 days to visit some friends and this would be a great place to come rest my head."

In [25]:
mendy[[item[0] for item in mendy].index(max([item[0] for item in mendy]))]

(26, 0.12486540101425797)

In [28]:
mendy[[item[1] for item in mendy].index(max([item[1] for item in mendy]))]

(15, 0.79818883615489977)

In [112]:
sum([item[1] for item in mendy])

0.99980718417481995

In [35]:
x = [1,2,3,4,13]
print x.index(1)
print x.index(max(x))
x[x.index(max(x))]

0
4


13

In [36]:
print x
x.append(100)
print x

[1, 2, 3, 4, 13]
[1, 2, 3, 4, 13, 100]


In [42]:
documents[192144]

'hi hugh,    my name is rani. this is my first time visiting redwood city for a meeting at stanford and was wondering if you had any availability on the night of march 22nd/23rd in your beautiful home. please let me know if i can provide any additional information in regards to the booking.    thank you!    best,  rani'

In [ ]:
#Hyper-parameter optimization: compare likelihood on test set for different number of topics

In [43]:
#make a list of message text, what topic was the most likely for that message text

random.seed(20170710)
#sample 10000 random contacts
n_corpus = len(corpus)
n_sample = 10000

mask = random.sample(xrange(n_corpus), n_sample)
messages_topics = []

#find the most relevant topic to each of these documents
for i in range(n_sample):
    #get array of topics and probabilities for document i
    results_i = lda_model_30[corpus[mask[i]]]
    
    ret = results_i[[item[1] for item in results_i].index(max([item[1] for item in results_i]))]
    
    messages_topics.append([documents[mask[i]], ret[0], ret[1]])
    
    
messages_topic_0 = [x for x in messages_topics if x[1] == 0]
messages_topic_1 = [x for x in messages_topics if x[1] == 1]
messages_topic_2 = [x for x in messages_topics if x[1] == 2]
messages_topic_3 = [x for x in messages_topics if x[1] == 3]
messages_topic_4 = [x for x in messages_topics if x[1] == 4]
messages_topic_5 = [x for x in messages_topics if x[1] == 5]

    

In [179]:
#sample messages tagged with topic 1, make a list of [message text, topic, probability of being in topic]
random.seed(20170713)

#sample 100000 random contacts
n_corpus = len(corpus)
n_sample = 100000

mask = random.sample(xrange(n_corpus), n_sample)
messages_topic_1 = []

#find all the documents that are tagged with topic 1
for i in range(n_sample):
    #get array of topics and probabilities for document i
    results_i = lda_model_30[corpus[mask[i]]]
    #make list of [topic 1, probability of being in topic 1]
    list_temp = [item for item in results_i if item[0] == 1]
    #check if the topic is actually tagged
    if len(list_temp) > 0:
        messages_topic_1.append([documents[mask[i]], list_temp[0][0], list_temp[0][1]])
        

    
    

In [182]:
[x for x in messages_topic_1 if x[2] > 0.5]

[['visiting / touring', 1, 0.50902800547025029],
 ['hello dino jose here', 1, 0.99855217174238342],
 ['masters', 1, 0.99710867397806613],
 ['we are prisca and stephanie. ', 1, 0.99710867397806613],
 ['my niece is graduating from cal state fullerton', 1, 0.5596125671132961],
 ['attending an accelerator', 1, 0.99855217174238342],
 ['hej justyna!:)    razem z mezem pawlem pochodzimy z wroclawia, gdzie zyjemy i pracujemy. przyjezdzamy z mezem do us pierwszy raz. 20 maja wracamy z miami i konczymy przygode w nowym yorku:) przylatujemy w okolicach 11 am, wiec pytanie czy moglibysmy sie zameldowac w okolicach 1 pm?    wracamy do polski 25 maja i lot mamy pozno bo o 9 pm. czy jest mozliwosc zostawienia bagazy abysmy mogli przed lotem przespacerowac sie jeszcze ostatni raz?:)    dziekuje z gory za odpowiedz i pozdrawiamy,  justyna & pawel',
  1,
  0.5842105274620164],
 ['state farm', 1, 0.99855217174238342],
 ['hi leida!', 1, 0.99855217174238342],
 ['heavyyyyyyyyyyy', 1, 0.99710867397806613],
 

In [159]:
y = lda_model_30[corpus[1]]

In [165]:
z = [item for item in y if item[0] == 2]

In [169]:
z[0][1]

0.041313581177095861

In [170]:
len(z)

1

In [192]:
#make a pandas dataframe that has the schema [message text, topic, topic probability]

df = pd.DataFrame([7, lda_model_30[corpus[7]]])



In [193]:
df

,0
0,7
1,"[(7, 0.100079848113), (9, 0.334752994663), (11..."


In [191]:
df.append(pd.DataFrame(lda_model_30[corpus[100]]))

,0,1
0,7,0.099578
1,9,0.334653
2,11,0.056714
3,21,0.446981
4,25,0.061956
0,19,0.169406
1,21,0.157008
2,28,0.673452


In [264]:
z = lda_model_30[corpus[17231]]
z0 = [x[0] for x in z]
z1 = [x[1] for x in z]
df = pd.DataFrame.from_items([('a', z0), ('b', z1)])

In [265]:
df

,a,b
0,9,0.141697
1,20,0.338097
2,26,0.519936


In [202]:
z0

[9, 20, 26]

In [226]:
df1 = pd.DataFrame([z0, z1, [1] * len(z0), ['meep'] * len(z0)]).transpose()

In [227]:
df1

,0,1,2,3
0,9,0.141637,1,meep
1,20,0.338185,1,meep
2,26,0.519908,1,meep


In [221]:
df1.append(pd.DataFrame([z0, z1]).transpose())

,0,1
0,9.0,0.141637
1,20.0,0.338185
2,26.0,0.519908
0,9.0,0.141637
1,20.0,0.338185
2,26.0,0.519908


In [45]:
#sample messages tagged with topic 1, make a list of [message text, topic, probability of being in topic]
random.seed(20170713)

#sample 10000 random contacts
n_corpus = len(corpus)
n_sample = 10000
mask = random.sample(xrange(n_corpus), n_sample)

#initialize dataframe
df = pd.DataFrame(data = None, columns = ['index', 'topic', 'probability', 'text'])



for i in range(n_sample):
    #get array of topics and probabilities for document i
    results_i = lda_model_30[corpus[mask[i]]]
    r0 = [x[0] for x in results_i]
    r1 = [x[1] for x in results_i]
    #append index (mask), topic, probability and text to dataframe
    #df.append(pd.DataFrame([mask[i]] * len(r0),r0, r1, [documents[mask[i]]] * len(r0)).transpose()) 
    df_temp = pd.DataFrame.from_items([('index', [mask[i]] * len(r0)), ('topic', r0), ('probability', r1), ('text', [documents[mask[i]]] * len(r0))])
    df = df.append(df_temp, ignore_index = True)

    



In [48]:
df.head()

,index,topic,probability,text
0,479585,7,0.435787,fishing and visit family
1,479585,19,0.563280,fishing and visit family
2,208567,2,0.094480,"hi, first time headed to chatanooga for a ..."
3,208567,9,0.204221,"hi, first time headed to chatanooga for a ..."
4,208567,14,0.140526,"hi, first time headed to chatanooga for a ..."


In [47]:
df.groupby('topic').agg('count').sort_values('text')

,index,probability,text
topic,,,
1,975,975,975
28,986,986,986
2,991,991,991
13,1062,1062,1062
9,1082,1082,1082
20,1085,1085,1085
14,1187,1187,1187
16,1219,1219,1219
11,1230,1230,1230


In [230]:
df = pd.DataFrame(data = None, columns = ['index', 'topic', 'probability', 'text'])

In [232]:
[1] * 8

[1, 1, 1, 1, 1, 1, 1, 1]

In [234]:
documents[1]

'hi sara! i am taking my parents and two kids to san diego for a week and would love to stay at your place! definitely need a quiet place for my parents, parking for our rental, and a place for my little one to sleep--looks like you have the entire package!!!!    let me know if it is available!  janellyn'

In [241]:
df.append(pd.DataFrame([z0, z1]).transpose())

,0,1,probability,topic
0,9.0,0.141616,NaN,NaN
1,20.0,0.338469,NaN,NaN
2,26.0,0.519645,NaN,NaN


In [242]:
df

,topic,probability


In [245]:
df

,topic,probability


In [6]:
from airpy.analysis import erf 
exp = erf.AdHocERFExperiment('web_first_message_suggestions')
exp_data = exp.metrics.contact_to_book.get_subjects().head()

presto: Query: Complete after 20.31 sec on 2017-09-14. CAVEATS: Loaded from cache.
presto: Query: Complete after 14.70 sec on 2017-09-14. CAVEATS: Loaded from cache.
presto: Query: Complete after 14.74 sec on 2017-09-14. CAVEATS: Loaded from cache.


In [12]:
exp_data.head()

assignment  bookings  contact_trips     ratio
dim_treatment id_subject                                               
control       10                 1.0       1.0            1.0  1.000000
              100000053          1.0       1.0            0.0       inf
              100000371          1.0       1.0            0.0       inf
              10000038           1.0       1.0            1.0  1.000000
              100000401          1.0       0.0            0.0       NaN